<a href="https://colab.research.google.com/github/scapemotes/Google-drive-tramsger/blob/main/_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<font color='blue'> Any File to Google Drive - Shared by [Cheems](https://github.com/cheems) </font>**


NOUNT

In [1]:
 
 
 #@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


---
#Mount Google Drive with Rclone

In [2]:
 #@markdown <h3>📝 Note: Run this before using Rclone.</h3>
#@markdown <h4>📝 Upload an rclone profile</4>
Setup_Time_Zone = False 
 
import os; from google.colab import files; from IPython.display import HTML, clear_output
 
def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Please upload the config file of rclone (rclone.conf) from your computer.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))
 
upload_conf()

---
# <img src='https://szyha.github.io/RcloneLabArchive/img/title_jdownloader.png' height="45" alt="JDownloader"/>

URL

In [3]:
 
 
 URL = "https://skyler.privatestorag.comq/dl/OgP4jLxcw765v8vgDuuqRw/1632262410/787487603/6115fd262c5c83.50406287/Ted.Lasso.S02E04.WEBRip.x264-ION10.mp4" #@param {type:"string"}
#@markdown <center><h5>Default output path: /content/downloads/</h5></center>
#@markdown <center><h5>For Google Drive output path: /content/drive/My Drive/</h5></center>
OUTPUT_PATH = "/content/drive/My Drive/" #@param {type:"string"}
 
import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ocr.py"): 
    from subprocess import run
    from shlex import split
 
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh
 
def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)
 
def aria2Install():
  runSh('apt install -y aria2')
 
def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()
 
 
    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)
 
 
    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False
 
        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return
 
        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)
 
        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL
 
if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/downloads/"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("downloads")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

URL https://skyler.privatestorag.comq/dl/OgP4jLxcw765v8vgDuuqRw/1632262410/787487603/6115fd262c5c83.50406287/Ted.Lasso.S02E04.WEBRip.x264-ION10.mp4
[generic] Ted.Lasso.S02E04.WEBRip.x264-ION10: Requesting header
[generic] Ted.Lasso.S02E04.WEBRip.x264-ION10: Downloading webpage
ERROR: Unable to download webpage: <urlopen error [Errno -2] Name or service not known> (caused by URLError(gaierror(-2, 'Name or service not known')))


# New Section

Url transfer

In [ ]:
 
 
 URL = "" #@param {type:"string"}
#@markdown <center><h5>Default output path: /content/downloads/</h5></center>
#@markdown <center><h5>For Google Drive output path: /content/drive/My Drive/</h5></center>
OUTPUT_PATH = "" #@param {type:"string"}
 
import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ocr.py"): 
    from subprocess import run
    from shlex import split
 
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh
 
def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)
 
def aria2Install():
  runSh('apt install -y aria2')
 
def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()
 
 
    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)
 
 
    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False
 
        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return
 
        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)
 
        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL
 
if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/downloads/"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("downloads")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

---
# <img src='https://szyha.github.io/RcloneLabArchive/img/title_rclonelab.png' height="45" alt="RcloneLab"/>

In [ ]:
 # ============================= FORM ============================= #
Mode = "Copy" #@param ["Off","Config", "Copy", "Move", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash", "qBittorrent"]
Compare = "Size & Mod-Time" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Source = "premium:/PSX/" #@param {type:"string"}
Destination = "melody:input" #@param {type:"string"}
Transfers = 15 #@param {type:"slider", min:1, max:20, step:1}
Checkers = 3 #@param {type:"slider", min:1, max:40, step:1}
#@markdown ---
 
#@markdown <center><h3><font color="#3399ff"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>
 
#@markdown ---
Simple_Ouput = True #@param {type:"boolean"}
Skip_files_that_are_newer_on_the_destination = True #@param {type:"boolean"}
Skip_all_files_that_exist = True #@param {type:"boolean"}
Do_not_cross_filesystem_boundaries = False
Do_not_update_modtime_if_files_are_identical = False #@param {type:"boolean"}
Large_amount_of_files_optimization = False
Google_Drive_optimization = True #@param {type:"boolean"}
Dry_Run = False #@param {type:"boolean"}
Output_Log_File = "DEBUG" #@param ["OFF", "NOTICE", "INFO", "ERROR", "DEBUG"]
Extra_Arguments = "" #@param {type:"string"}
#@markdown ---
 
#@markdown <center><h3><font color="#3399ff"><b>↪️ Sync Configuration ↩️</b></font></h3></center>
 
#@markdown ---
Sync_Mode = "Delete during transfer" #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}
#@markdown ---
 
 
#@markdown <center><h3><font color="#3399ff"><b>💞 Deduplicate Configuration 💞</b></font></h3></center>
 
#@markdown ---
Deduplicate_Mode = "Interactive" #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = True #@param {type:"boolean"}
# ================================================================ #
 
import os; from IPython.display import HTML, clear_output
import sys
 
 
if Mode != "Off":
 
  # Optimized for Google Colaboratory
  os.environ["bufferC"] = "--buffer-size 96M"
 
  if Compare == "Size & Checksum":
    os.environ["compareC"] = "-c"
  elif Compare == "Only Mod-Time":
    os.environ["compareC"] = "--ignore-size"
  elif Compare == "Only Size":
    os.environ["compareC"] = "--size-only"
  elif Compare == "Only Checksum":
    os.environ["compareC"] = "-c --ignore-size"
  else:
    os.environ["compareC"] = ""
 
  os.environ["sourceC"] = Source
  os.environ["destinationC"] = Destination
  os.environ["transfersC"] = "--transfers "+str(Transfers)
  os.environ["checkersC"] = "--checkers "+str(Checkers)
 
  if Skip_files_that_are_newer_on_the_destination == True:
    os.environ["skipnewC"] = "-u"
  else:
    os.environ["skipnewC"] = ""
 
  if Skip_all_files_that_exist == True:
    os.environ["skipexistC"] = "--ignore-existing"
  else:
    os.environ["skipexistC"] = ""
 
  if Do_not_cross_filesystem_boundaries == True:
    os.environ["nocrossfilesystemC"] = "--one-file-system"
  else:
    os.environ["nocrossfilesystemC"] = ""
 
  if Do_not_update_modtime_if_files_are_identical == True:
    os.environ["noupdatemodtimeC"] = "--no-update-modtime"
  else:
    os.environ["noupdatemodtimeC"] = ""
 
  if Large_amount_of_files_optimization == True:
    os.environ["filesoptimizeC"] = "--fast-list"
  else:
    os.environ["filesoptimizeC"] = ""
 
  if Google_Drive_optimization == True:
    os.environ["driveoptimizeC"] = "--drive-chunk-size 32M --drive-acknowledge-abuse --drive-keep-revision-forever --disable copy"
  else:
    os.environ["driveoptimizeC"] = ""
 
  if Dry_Run == True:
    os.environ["dryrunC"] = "-n"
  else:
    os.environ["dryrunC"] = ""
 
  if Output_Log_File != "OFF":
    os.environ["statsC"] = "--log-file=/root/.rclone_log/rclone_log.txt"
  else:
    if Simple_Ouput == True:
      os.environ["statsC"] = "-v --stats-one-line --stats=5s"
    else:
      os.environ["statsC"] = "-v --stats=5s"
 
  if Output_Log_File == "INFO":
    os.environ["loglevelC"] = "--log-level INFO"
  elif Output_Log_File == "ERROR":
    os.environ["loglevelC"] = "--log-level ERROR"
  elif Output_Log_File == "DEBUG":
    os.environ["loglevelC"] = "--log-level DEBUG"
  else:
    os.environ["loglevelC"] = ""
 
  os.environ["extraC"] = Extra_Arguments
 
  if Sync_Mode == "Delete during transfer":
    os.environ["syncmodeC"] = "--delete-during"
  elif Sync_Mode == "Delete before transfering":
    os.environ["syncmodeC"] = "--delete-before"
  elif Sync_Mode == "Delete after transfering":
    os.environ["syncmodeC"] = "--delete-after"
 
  if Track_Renames == True:
    os.environ["trackrenamesC"] = "--track-renames"
  else:
    os.environ["trackrenamesC"] = ""
 
  if Deduplicate_Mode == "Interactive":
    os.environ["deduplicateC"] = "interactive"
  elif Deduplicate_Mode == "Skip":
    os.environ["deduplicateC"] = "skip"
  elif Deduplicate_Mode == "First":
    os.environ["deduplicateC"] = "first"
  elif Deduplicate_Mode == "Newest":
    os.environ["deduplicateC"] = "newest"
  elif Deduplicate_Mode == "Oldest":
    os.environ["deduplicateC"] = "oldest"
  elif Deduplicate_Mode == "Largest":
    os.environ["deduplicateC"] = "largest"
  elif Deduplicate_Mode == "Rename":
    os.environ["deduplicateC"] = "rename"
 
  if Deduplicate_Use_Trash == True:
    os.environ["deduplicatetrashC"] = ""
  else:
    os.environ["deduplicatetrashC"] = "--drive-use-trash=false"
 
  ### rclone Execution
 
  if Output_Log_File != "OFF" and Mode != "Config":
    !mkdir -p -m 666 /root/.rclone_log/
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enables, rclone will not output log through the terminal, please wait until finished.</h2><br></center>"))
 
 
  if Mode == "Config":
    !rclone --config=/root/.rclone.conf config
  elif Mode == "Copy":
    !rclone --config=/root/.rclone.conf copy "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Move":
    !rclone --config=/root/.rclone.conf move "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Sync":
    !rclone --config=/root/.rclone.conf sync "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Checker":
    !rclone --config=/root/.rclone.conf check "$sourceC" "$destinationC" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Deduplicate":
    !rclone --config=/root/.rclone.conf dedupe "$sourceC" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Remove Empty Directories":
    !rclone --config=/root/.rclone.conf rmdirs "$sourceC" $statsC $loglevelC $dryrunC $extraC
  elif Mode == "Empty Trash":
    !rclone --config=/root/.rclone.conf cleanup "$sourceC" $statsC $loglevelC $dryrunC $extraC
  elif Mode == "qBittorrent":
    !chmod -R 666 /content/qBittorrent/
    !rclone --config=/root/.rclone.conf move "/content/qBittorrent/" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs --exclude **/.unwanted/ $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
 
  ### Log Output
 
  if Output_Log_File != "OFF" and Mode != "Config":
    ### Rename log file and output settings.
    !mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
    with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
      f.write("Mode: " + Mode + \
              "\nCompare: " + Compare + \
              "\nSource: \"" + Source + \
              "\"\nDestination: \"" + Destination + \
              "\"\nTransfers: " + str(Transfers) + \
              "\nCheckers: " + str(Checkers) + \
              "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
              "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
              "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
              "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
              "\nDry-Run: " + str(Dry_Run) + \
              "\nOutput Log Level: " + Output_Log_File + \
              "\nExtra Arguments: \"" + Extra_Arguments + \
              "\"\nSync Moden: " + Sync_Mode + \
              "\nTrack Renames: " + str(Track_Renames) + \
              "\nDeduplicate Mode: " + Deduplicate_Mode + \
              "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))
    ### Compressing log file.
    !rm -f /root/rclone_log.zip
    !zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/
    !rm -rf /root/.rclone_log/
    !mkdir -p -m 666 /root/.rclone_log/
    ### Send Log
    if os.path.isfile("/root/rclone_log.zip") == True:
      try:
        files.download("/root/rclone_log.zip")
        !rm -f /root/rclone_log.zip
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Sending log to your browser...</h2><br></center>"))
      except:
        !mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">You can use file explorer to download the log file.</h2><br></center>"))
    else:
      clear_output(wait=True)
      display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))
 
  ### Operation has been successfully completed.
  if os.path.isfile("/usr/bin/rclone") == False:
    clear_output(wait=True)
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ Please run the installation cell above first.</h2><br></center>"))
  elif Mode != "Config":
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))
else:
  !echo "doing nothing because the mode is set to off"

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 📚 Follow me on GitHub for more: [_@cheems_](https://github.com/cheems)
# 📦 Repo: [Github](https://github.com/cheems/Any-file-to-Google-Drive)